In [1]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
def order_book(month,day):
    data1 = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/stat_order_book/CNF14_0'+str(month)+'_'+str(day)+'_order_book_final.csv'
    data1 = pd.read_csv(datapath,sep='\t',encoding = 'utf-8')
    data_book = data1[['0','1','2','3']]
    return data_book

In [3]:
def day_time(month,day):
    data = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.0'+str(month)+'.'+str(day)+'.csv'
    data = pd.read_csv(datapath)
    data_CNF14 = data[data.Series == 'CNF14']
    data = data_CNF14
    market_open_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D09:00')].index.tolist()[0] 
    market_close_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D16:00')].index.tolist()[0] 
    data_open = data[market_open_time:market_close_time + 1]
    timestamp_ = data_open.TimeStamp.unique()
    return timestamp_

In [4]:
def time_transform(timestamp_time):
    time_second = []
    for i in range(0,len(timestamp_time),1):
        second = float(timestamp_time[i][11])*36000 + float(timestamp_time[i][12])*3600 \
                +float(timestamp_time[i][14])*600 + float(timestamp_time[i][15])*60\
                +float(timestamp_time[i][17])*10 + float(timestamp_time[i][18])  
        time_second.append(second - 32400.0)
    return  time_second

In [5]:
def bid123_ask123_Q(data_book_28_open):
    Bid1 = []
    Bid2 = []
    Bid3 = []
    Bid1_Quantity = []
    Bid2_Quantity = []
    Bid3_Quantity = []
    Ask1 = []
    Ask2 = []
    Ask3 = []
    Ask1_Quantity = []
    Ask2_Quantity = []
    Ask3_Quantity = []

    TimeStamp = []
    for i in range(1,len(data_book_28_open),4):
        #print data_book_28_open.iloc[i]['0']
        #print data_book_28_open.iloc[i]['2']
        Bid1.append(float(data_book_28_open.iloc[i]['0'])/100.0)
        Bid1_Quantity.append(float(data_book_28_open.iloc[i]['1']))

        Bid2.append(float(data_book_28_open.iloc[i + 1]['0'])/100.0)
        Bid2_Quantity.append(float(data_book_28_open.iloc[i + 1]['1']))  

        Bid3.append(float(data_book_28_open.iloc[i + 2]['0'])/100.0)
        Bid3_Quantity.append(float(data_book_28_open.iloc[i + 2]['1']))  

        Ask1.append(float(data_book_28_open.iloc[i]['2'])/100.0) 
        Ask1_Quantity.append(float(data_book_28_open.iloc[i]['3']))

        Ask2.append(float(data_book_28_open.iloc[i + 1]['2'])/100.0) 
        Ask2_Quantity.append(float(data_book_28_open.iloc[i + 1]['3']))    

        Ask3.append(float(data_book_28_open.iloc[i + 2]['2'])/100.0) 
        Ask3_Quantity.append(float(data_book_28_open.iloc[i + 2]['3']))    
        TimeStamp.append(data_book_28_open.iloc[i-1][1])
    return Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity,    

In [6]:
def rise_ask(Ask1,timestamp_time_second):
    rise_ratio = []
    index = np.where(np.array(timestamp_time_second) >= 600)[0][0]
    for i in range(0,index):
        rise_ratio_ = round((Ask1[i] - Ask1[0])*(1.0)/Ask1[0]*100,5)
        rise_ratio.append(rise_ratio_)
    for i in range(index,len(Ask1),1):
        #print timestamp_time_second[:i]
        #print timestamp_time_second[i] - 600
        #print np.where(np.array(timestamp_time_second[:i]) >= timestamp_time_second[i] - 600)[0][0]
        index_start = np.where(np.array(timestamp_time_second[:i]) >= timestamp_time_second[i] - 600)[0][0]
        rise_ratio_ = round((Ask1[i] - Ask1[index_start])*(1.0)/Ask1[index_start]*100,5)
        rise_ratio.append(rise_ratio_)
    return rise_ratio


In [7]:
def rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,close_price,\
             N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,\
             N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
             N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI):
    
    if rise_ratio_ask[index] <= -0.4:
        N_1 = N_1 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            N_1_T = N_1_T + 1
            N_1_E = N_1_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_1_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_1_M = N_1_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_1_TI = N_1_TI + (time_traded - time_start)
            #print N_1_TI
            pass

        else:
            if i < 25200 - 900:
                N_1_E = N_1_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_1_M = N_1_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                #print N_1_M
                N_1_TI = N_1_TI + (900.0)
                #print N_1_TI
            else:
                N_1_E = N_1_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_1_M = N_1_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0) 
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_1_TI = N_1_TI + (time_traded - time_start)
                #print N_1_TI
            pass

    elif -0.4 < rise_ratio_ask[index] <= -0.15:
        N_2 = N_2 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 3
            N_2_T = N_2_T + 1
            N_2_E = N_2_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_2_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_2_M = N_2_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_2_TI = N_2_TI + (time_traded - time_start)
            #print N_2_TI
            pass
        else:
            if i < 25200 - 900:
                N_2_E = N_2_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_2_M = N_2_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_2_TI = N_2_TI + (900.0)
                #print N_2_TI
            else:
                N_2_E = N_2_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_2_M = N_3_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_2_TI = N_2_TI + (time_traded - time_start)
                #print N_2_TI
            pass

    elif -0.15 < rise_ratio_ask[index] <= 0.15:
        N_3 = N_3 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 5
            N_3_T = N_3_T + 1
            N_3_E = N_3_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_3_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_3_M = N_3_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_3_TI = N_3_TI + (time_traded - time_start)
            #print N_3_TI
            pass
        else:
            if i < 25200 - 900:
                N_3_E = N_3_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_3_M = N_3_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_3_TI = N_3_TI + (900.0)
                #print N_3_TI
            else:
                N_3_E = N_3_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_3_M = N_3_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_3_TI = N_3_TI + (time_traded - time_start)
                #print N_3_TI
            pass

    elif 0.15 < rise_ratio_ask[index] < 0.4:
        N_4 = N_4 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            #print 7
            N_4_T = N_4_T + 1
            N_4_E = N_4_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_4_E
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_4_M = N_4_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_4_TI = N_4_TI + (time_traded - time_start)
            #print N_4_TI
            pass
        else:
            if i < 25200 - 900:
                N_4_E = N_4_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_4_M = N_4_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_4_TI = N_4_TI + (900.0)
                #print N_4_TI
            else:
                N_4_E = N_4_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_4_M = N_4_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_4_TI = N_4_TI + (time_traded - time_start)
                #print N_4_TI
            pass

    elif 0.4 <= rise_ratio_ask[index]:
        N_5 = N_5 + 1
        #print rise_ratio_ask[index]
        if Bid[index] > min(Ask[(index+1):index + 1 + index_15_min + 1]):
            N_5_T = N_5_T + 1
            N_5_E = N_5_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_5_E      
            first_min_index = np.where(np.array(Ask[(index+1):index + 1 + index_15_min + 1]) < Bid[index])[0][0]
            N_5_M = N_5_M + (max(Ask[(index + 1):(index + 1 + first_min_index + 1)]) - Bid[index])*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_min_index + 1]
            N_5_TI = N_5_TI + (time_traded - time_start)
            #print N_5_TI
            pass
        else:
            if i < 25200 - 900:
                N_5_E = N_5_E + (Ask[index] - Ask[index + index_15_min])*(1.0)/(5.0)
                N_5_M = N_5_M + (max(Ask[(index + 1):(index + 1 + index_15_min + 1)]) - Bid[index])*(1.0)/(5.0) 
                N_1_TI = N_1_TI + (900.0)
                #print N_5_TI
            else:
                N_5_E = N_5_E + (Ask[index] - close_price)*(1.0)/(5.0)
                N_5_M = N_5_M + (max(Ask[(index+1):]) - Bid[index])*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_5_TI = N_5_TI + (time_traded - time_start)
                #print N_5_TI 
            pass
        
    return N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
           N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI


In [8]:
def bid(time1,time2,Bid,Ask,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,\
        Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weigth1,weigth2,weigth3):
    [N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
     N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI] = [0 for i in  range(0,25,1)]
    [N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
     N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI] = [0 for i in  range(0,25,1)]
    [N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
     N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI] = [0 for i in  range(0,25,1)]
    [N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
     N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI] = [0 for i in  range(0,25,1)]
    [N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
     N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI] = [0 for i in  range(0,25,1)]

    for i in range(time1,time2,1):
        #print i
        #print np.where(array(timestamp_time_second) <= i)[0]
        index = np.where(array(timestamp_time_second) <= i)[0][-1]
        #print i
        #print index
        #print timestamp_time_second[index]
        if i < 25200 - 900:
            index_15_min = np.where(np.array(timestamp_time_second[index:]) >= i + 900)[0][0]
            #print index_15_min
            #print timestamp_time_second[index+index_15_min]
        #print 'time stamp = %d'%(timestamp_time_second[index])
        #print index
        #print index + index_15_min 
        #rint Bid1[index]
        #print Ask1[index]
        #print len(Ask1[(index+1):index + index_15_min])

        #if Bid1[index] > min(Ask1[(index+1):index + index_15_min]):
        #    first_min_index = np.where(np.array(Ask1[(index+1):index + index_15_min]) < Bid1[index])[0][0]
        #    print 'mdd = %d'%((max(Ask1[(index + 1):(index + 1 + first_min_index)]) - Bid1[index])*(1.0)/(5.0))
        #print min_Ask1_time_series[index + 1]
        weighted_Bid = Bid1_Quantity[index] * weight1 + Bid2_Quantity[index] * weight2 + Bid3_Quantity[index] * weight3
        weighted_Ask = Ask1_Quantity[index] * weight1 + Ask2_Quantity[index] * weight2 + Ask3_Quantity[index] * weight3  
        #print weighted_Bid 
        #print weighted_Ask
        #print (weighted_Bid*(1.0) / weighted_Ask)

        if (weighted_Bid*(1.0) / weighted_Ask) <= 0.7:
            N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
            N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,\
                                                          N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,\
                                                          N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI)                                               

        elif 0.7 < (weighted_Bid*(1.0) / weighted_Ask) <= 0.9:
            N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
            N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,\
                                                          N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,\
                                                          N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI)                                                

        elif 0.9 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.1:
            N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
            N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,\
                                                          N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,\
                                                          N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI)                                    

        elif 1.1 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.3:
            N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
            N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,\
                                                          N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,\
                                                          N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI)                              

        elif 1.30 < (weighted_Bid*(1.0) / weighted_Ask):
            N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
            N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI = rise_cal(i,rise_ratio_ask,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,\
                                                          N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,\
                                                          N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI)                 

    return [[N_11,N_12,N_13,N_14,N_15],[N_11_T,N_12_T,N_13_T,N_14_T,N_15_T],[N_11_E,N_12_E,N_13_E,N_14_E,N_15_E],\
             [N_11_M,N_12_M,N_13_M,N_14_M,N_15_M],[N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI],\
            [N_21,N_22,N_23,N_24,N_25],[N_21_T,N_22_T,N_23_T,N_24_T,N_25_T],[N_21_E,N_22_E,N_23_E,N_24_E,N_25_E],\
             [N_21_M,N_22_M,N_23_M,N_24_M,N_25_M],[N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI],\
            [N_31,N_32,N_33,N_34,N_35],[N_31_T,N_32_T,N_33_T,N_34_T,N_35_T],[N_31_E,N_32_E,N_33_E,N_34_E,N_35_E],\
             [N_31_M,N_32_M,N_33_M,N_34_M,N_35_M],[N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI],\
            [N_41,N_42,N_43,N_44,N_45],[N_41_T,N_42_T,N_43_T,N_44_T,N_45_T],[N_41_E,N_42_E,N_43_E,N_44_E,N_45_E],\
             [N_41_M,N_42_M,N_43_M,N_44_M,N_45_M],[N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI],\
            [N_51,N_52,N_53,N_54,N_55],[N_51_T,N_52_T,N_53_T,N_54_T,N_55_T],[N_51_E,N_52_E,N_53_E,N_54_E,N_55_E],\
             [N_51_M,N_52_M,N_53_M,N_54_M,N_55_M],[N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI]] 


### day 1/15

In [9]:
data_book = order_book(1,15)
data_book_open = data_book[1108:275559+1]
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,15)
timestamp_time_second = time_transform(timestamp_time)

In [11]:
#data_book_open

In [10]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 114.778342


In [11]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 83.009253


9:00~9:15

In [12]:
close_price = 6740
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [13]:
time1 = 0
time2 = 600
mat28_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

9:15~11:00

In [14]:
time1 = 600
time2 = 7200
mat28_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

11:00~13:00

In [15]:
time1 = 7200
time2 = 14400
mat28_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

13:00~16:00

In [16]:
time1 = 14400
time2 = 25200
mat28_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [24]:
#mat28_3_15

### Day 1/14

In [25]:
data_book = order_book(1,14)
data_book_open = data_book[880:336307+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,14)
timestamp_time_second = time_transform(timestamp_time)

In [14]:
#data_book_open

In [26]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 136.307194


In [27]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 126.092943


In [28]:
close_price = 6780
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [29]:
time1 = 0
time2 = 600
mat29_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [30]:
time1 = 600
time2 = 7200
mat29_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [31]:
time1 = 7200
time2 = 14400
mat29_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [32]:
time1 = 14400
time2 = 25200
mat29_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [42]:
#mat29_1_15

### Day 1/13

In [43]:
data_book = order_book(1,13)
data_book_open = data_book[1136:331583+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,13)
timestamp_time_second = time_transform(timestamp_time)

In [44]:
#data_book_open
#len(timestamp_time_second)

In [45]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 134.038121


In [46]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 127.971429


In [47]:
close_price = 6740
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [48]:
time1 = 0
time2 = 600
mat27_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [49]:
time1 = 600
time2 = 7200
mat27_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [50]:
time1 = 7200
time2 = 14400
mat27_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [51]:
time1 = 14400
time2 = 25200
mat27_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/10

In [52]:
data_book = order_book(1,10)
data_book_open = data_book[904:351239+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,10)
timestamp_time_second = time_transform(timestamp_time)

In [24]:
#data_book_open

In [22]:
len(timestamp_time_second)

87584

In [53]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 142.267832


In [83]:
len(Ask1)

82556

In [54]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 332.848180


In [55]:
close_price = 6760
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [56]:
time1 = 0
time2 = 600
mat24_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [57]:
time1 = 600
time2 = 7200
mat24_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [58]:
time1 = 7200
time2 = 14400
mat24_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [59]:
time1 = 14400
time2 = 25200
mat24_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [66]:
#mat24_3_11

### Day 1/09

In [67]:
data_book = order_book(1,'09')
data_book_open = data_book[856:366355+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'09')
timestamp_time_second = time_transform(timestamp_time)

In [32]:
#data_book_open

In [68]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 162.233027


In [69]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 356.347721


In [70]:
close_price = 6770
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [71]:
time1 = 0
time2 = 600
mat22_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [72]:
time1 = 600
time2 = 7200
mat22_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [73]:
time1 = 7200
time2 = 14400
mat22_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [74]:
time1 = 14400
time2 = 25200
mat22_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/08

In [75]:
data_book = order_book(1,'08')
data_book_open = data_book[1236:356399+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'08')
timestamp_time_second = time_transform(timestamp_time)

In [37]:
#data_book_open

In [76]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 119.563197


In [77]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 283.613185


In [78]:
close_price = 6810
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [79]:
time1 = 0
time2 = 600
mat21_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [80]:
time1 = 600
time2 = 7200
mat21_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [81]:
time1 = 7200
time2 = 14400
mat21_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [82]:
time1 = 14400
time2 = 25200
mat21_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/07

In [83]:
data_book = order_book(1,'07')
data_book_open = data_book[1040:325879+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'07')
timestamp_time_second = time_transform(timestamp_time)

In [40]:
#data_book_open

In [84]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 109.099198


In [85]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 335.189806


In [86]:
close_price = 6790
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [87]:
time1 = 0
time2 = 600
mat20_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [88]:
time1 = 600
time2 = 7200
mat20_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [89]:
time1 = 7200
time2 = 14400
mat20_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [90]:
time1 = 14400
time2 = 25200
mat20_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/06

In [91]:
data_book = order_book(1,'06')
data_book_open = data_book[800:332843+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'06')
timestamp_time_second = time_transform(timestamp_time)

In [92]:
#data_book_open

In [93]:
import time 

start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 125.940732


In [94]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 364.051424


In [95]:
close_price = 6806
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [96]:
time1 = 0
time2 = 600
mat17_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [97]:
time1 = 600
time2 = 7200
mat17_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [98]:
time1 = 7200
time2 = 14400
mat17_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [99]:
time1 = 14400
time2 = 25200
mat17_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/03

In [100]:
data_book = order_book(1,'03')
data_book_open = data_book[984:318387+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'03')
timestamp_time_second = time_transform(timestamp_time)

In [46]:
#data_book_open

In [101]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 106.708166


In [102]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 303.926843


In [103]:
close_price = 6915
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [104]:
time1 = 0
time2 = 600
mat16_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [105]:
time1 = 600
time2 = 7200
mat16_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [106]:
time1 = 7200
time2 = 14400
mat16_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [107]:
time1 = 14400
time2 = 25200
mat16_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/02

In [108]:
data_book = order_book(1,'02')
data_book_open = data_book[1172:234835+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'02')
timestamp_time_second = time_transform(timestamp_time)

In [49]:
#data_book_open

In [109]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 78.434215


In [110]:
import time 
start = time.time()
rise_ratio_ask = rise_ask(Ask1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 171.014962


In [111]:
close_price = 7030
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [112]:
time1 = 0
time2 = 600
mat02_1_15 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_2_15 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_3_15 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [113]:
time1 = 600
time2 = 7200
mat02_1_11 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_2_11 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_3_11 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [114]:
time1 = 7200
time2 = 14400
mat02_1_13 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_2_13 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_3_13 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [115]:
time1 = 14400
time2 = 25200
mat02_1_16 = bid(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_2_16 = bid(time1,time2,Bid2,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_3_16 = bid(time1,time2,Bid3,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [116]:
SUM_Bid1_15 = \
array(mat02_1_15)+array(mat16_1_15)+array(mat17_1_15)+array(mat20_1_15)+array(mat21_1_15)+array(mat22_1_15)+\
array(mat24_1_15)+array(mat27_1_15)+array(mat28_1_15)+array(mat29_1_15)
SUM_Bid2_15 = \
array(mat02_2_15)+array(mat16_2_15)+array(mat17_2_15)+array(mat20_2_15)+array(mat21_2_15)+array(mat22_2_15)+\
array(mat24_2_15)+array(mat27_2_15)+array(mat28_2_15)+array(mat29_2_15)
SUM_Bid3_15 = \
array(mat02_3_15)+array(mat16_3_15)+array(mat17_3_15)+array(mat20_3_15)+array(mat21_3_15)+array(mat22_3_15)+\
array(mat24_3_15)+array(mat27_3_15)+array(mat28_3_15)+array(mat29_3_15)

In [117]:
SUM_Bid1_11 = \
array(mat02_1_11)+array(mat16_1_11)+array(mat17_1_11)+array(mat20_1_11)+array(mat21_1_11)+array(mat22_1_11)+\
array(mat24_1_11)+array(mat27_1_11)+array(mat28_1_11)+array(mat29_1_11)
SUM_Bid2_11 = \
array(mat02_2_11)+array(mat16_2_11)+array(mat17_2_11)+array(mat20_2_11)+array(mat21_2_11)+array(mat22_2_11)+\
array(mat24_2_11)+array(mat27_2_11)+array(mat28_2_11)+array(mat29_2_11)
SUM_Bid3_11 = \
array(mat02_3_11)+array(mat16_3_11)+array(mat17_3_11)+array(mat20_3_11)+array(mat21_3_11)+array(mat22_3_11)+\
array(mat24_3_11)+array(mat27_3_11)+array(mat28_3_11)+array(mat29_3_11)

In [118]:
SUM_Bid1_13 = \
array(mat02_1_13)+array(mat16_1_13)+array(mat17_1_13)+array(mat20_1_13)+array(mat21_1_13)+array(mat22_1_13)+\
array(mat24_1_13)+array(mat27_1_13)+array(mat28_1_13)+array(mat29_1_13)
SUM_Bid2_13 = \
array(mat02_2_13)+array(mat16_2_13)+array(mat17_2_13)+array(mat20_2_13)+array(mat21_2_13)+array(mat22_2_13)+\
array(mat24_2_13)+array(mat27_2_13)+array(mat28_2_13)+array(mat29_2_13)
SUM_Bid3_13 = \
array(mat02_3_13)+array(mat16_3_13)+array(mat17_3_13)+array(mat20_3_13)+array(mat21_3_13)+array(mat22_3_13)+\
array(mat24_3_13)+array(mat27_3_13)+array(mat28_3_13)+array(mat29_3_13)

In [119]:
SUM_Bid1_16 = \
array(mat02_1_16)+array(mat16_1_16)+array(mat17_1_16)+array(mat20_1_16)+array(mat21_1_16)+array(mat22_1_16)+\
array(mat24_1_16)+array(mat27_1_16)+array(mat28_1_16)+array(mat29_1_16)
SUM_Bid2_16 = \
array(mat02_2_16)+array(mat16_2_16)+array(mat17_2_16)+array(mat20_2_16)+array(mat21_2_16)+array(mat22_2_16)+\
array(mat24_2_16)+array(mat27_2_16)+array(mat28_2_16)+array(mat29_2_16)
SUM_Bid3_16 = \
array(mat02_3_16)+array(mat16_3_16)+array(mat17_3_16)+array(mat20_3_16)+array(mat21_3_16)+array(mat22_3_16)+\
array(mat24_3_16)+array(mat27_3_16)+array(mat28_3_16)+array(mat29_3_16)

In [120]:
SUM_Bid1_15

array([[  3.07000000e+02,   3.23000000e+02,   3.18500000e+03,
          2.96000000e+02,   2.20000000e+01],
       [  2.13000000e+02,   2.89000000e+02,   2.93700000e+03,
          2.67000000e+02,   2.20000000e+01],
       [ -4.30000000e+01,   2.54000000e+02,   3.12800000e+03,
          2.56000000e+02,   2.20000000e+01],
       [  9.08000000e+02,   5.56000000e+02,   7.10800000e+03,
          4.71000000e+02,   2.20000000e+01],
       [  1.08704000e+05,   1.10766000e+05,   1.29522200e+06,
          5.71660000e+04,   2.17400000e+03],
       [  3.95000000e+02,   5.01000000e+02,   9.77000000e+02,
          1.19000000e+02,   4.00000000e+00],
       [  3.47000000e+02,   4.38000000e+02,   9.04000000e+02,
          1.17000000e+02,   4.00000000e+00],
       [  2.26000000e+02,   3.55000000e+02,   9.41000000e+02,
          1.56000000e+02,   4.00000000e+00],
       [  9.02000000e+02,   1.01800000e+03,   2.27400000e+03,
          2.62000000e+02,   8.00000000e+00],
       [  1.03971000e+05,   1.5072900

In [121]:
SUM_Bid2_15

array([[  3.07000000e+02,   3.23000000e+02,   3.18500000e+03,
          2.96000000e+02,   2.20000000e+01],
       [  2.13000000e+02,   2.78000000e+02,   2.83200000e+03,
          2.57000000e+02,   2.20000000e+01],
       [  1.70000000e+02,   5.01000000e+02,   5.97600000e+03,
          5.12000000e+02,   4.40000000e+01],
       [  1.22500000e+03,   9.43000000e+02,   1.03780000e+04,
          8.35000000e+02,   4.40000000e+01],
       [  1.27318000e+05,   1.34770000e+05,   1.62482600e+06,
          8.85820000e+04,   4.00000000e+03],
       [  3.95000000e+02,   5.01000000e+02,   9.77000000e+02,
          1.19000000e+02,   4.00000000e+00],
       [  3.45000000e+02,   4.10000000e+02,   8.42000000e+02,
          1.14000000e+02,   4.00000000e+00],
       [  5.63000000e+02,   7.18000000e+02,   1.75100000e+03,
          2.73000000e+02,   8.00000000e+00],
       [  1.33600000e+03,   1.67200000e+03,   3.32600000e+03,
          3.94000000e+02,   1.20000000e+01],
       [  1.40314000e+05,   2.0065400

In [122]:
SUM_Bid3_15

array([[  3.07000000e+02,   3.23000000e+02,   3.18500000e+03,
          2.96000000e+02,   2.20000000e+01],
       [  2.11000000e+02,   1.54000000e+02,   2.22500000e+03,
          2.57000000e+02,   2.20000000e+01],
       [  3.76000000e+02,   3.82000000e+02,   7.32200000e+03,
          7.69000000e+02,   6.60000000e+01],
       [  1.55200000e+03,   1.63800000e+03,   1.37680000e+04,
          1.13600000e+03,   6.60000000e+01],
       [  1.47033000e+05,   2.13728000e+05,   1.96117000e+06,
          1.17549000e+05,   4.00000000e+03],
       [  3.95000000e+02,   5.01000000e+02,   9.77000000e+02,
          1.19000000e+02,   4.00000000e+00],
       [  3.41000000e+02,   2.27000000e+02,   7.38000000e+02,
          1.14000000e+02,   4.00000000e+00],
       [  8.88000000e+02,   3.91000000e+02,   2.45000000e+03,
          3.87000000e+02,   1.20000000e+01],
       [  1.75500000e+03,   2.71500000e+03,   4.38300000e+03,
          5.57000000e+02,   1.60000000e+01],
       [  1.60018000e+05,   3.2898200

In [123]:
SUM_Bid1_11

array([[  6.81000000e+02,   2.27200000e+03,   5.03200000e+03,
          8.14000000e+02,   5.97000000e+02],
       [  6.11000000e+02,   1.99700000e+03,   4.29500000e+03,
          6.88000000e+02,   4.85000000e+02],
       [  4.38000000e+02,   1.45000000e+03,   2.34800000e+03,
          4.50000000e+02,   2.79000000e+02],
       [  1.36800000e+03,   4.89900000e+03,   1.30230000e+04,
          2.67600000e+03,   2.21900000e+03],
       [  3.06597000e+05,   5.72221000e+05,   1.48155600e+06,
          2.85690000e+05,   7.91150000e+04],
       [  6.71000000e+02,   2.38300000e+03,   6.59000000e+03,
          1.12700000e+03,   6.71000000e+02],
       [  5.30000000e+02,   1.93700000e+03,   5.24200000e+03,
          8.36000000e+02,   5.22000000e+02],
       [ -1.45000000e+02,   7.05000000e+02,   1.67700000e+03,
          1.24000000e+02,   2.85000000e+02],
       [  2.24100000e+03,   6.67200000e+03,   1.88450000e+04,
          3.88100000e+03,   2.79100000e+03],
       [  4.10603000e+05,   8.9294700

In [124]:
SUM_Bid2_11

array([[  6.81000000e+02,   2.27200000e+03,   5.03200000e+03,
          8.14000000e+02,   5.97000000e+02],
       [  4.64000000e+02,   1.79300000e+03,   3.80600000e+03,
          6.09000000e+02,   4.63000000e+02],
       [  7.10000000e+02,   2.58400000e+03,   5.36200000e+03,
          8.59000000e+02,   6.97000000e+02],
       [  2.49600000e+03,   7.99600000e+03,   1.97880000e+04,
          3.79400000e+03,   3.03700000e+03],
       [  4.57018000e+05,   9.63459000e+05,   2.19073500e+06,
          3.86826000e+05,   1.40097000e+05],
       [  6.71000000e+02,   2.38300000e+03,   6.59000000e+03,
          1.12700000e+03,   6.71000000e+02],
       [  3.87000000e+02,   1.58300000e+03,   4.33500000e+03,
          6.36000000e+02,   4.83000000e+02],
       [ -3.45000000e+02,   1.15400000e+03,   4.33800000e+03,
          5.28000000e+02,   7.12000000e+02],
       [  3.67200000e+03,   1.03370000e+04,   2.78290000e+04,
          5.35300000e+03,   3.64300000e+03],
       [  5.88769000e+05,   1.3572220

In [125]:
SUM_Bid3_11

array([[  6.81000000e+02,   2.27200000e+03,   5.03200000e+03,
          8.14000000e+02,   5.97000000e+02],
       [  3.42000000e+02,   1.41900000e+03,   2.90800000e+03,
          5.33000000e+02,   4.26000000e+02],
       [  1.05400000e+03,   3.21700000e+03,   6.64000000e+03,
          1.31300000e+03,   1.10200000e+03],
       [  3.21200000e+03,   1.09100000e+04,   2.59970000e+04,
          4.66500000e+03,   3.64600000e+03],
       [  5.45745000e+05,   1.32030700e+06,   2.96199800e+06,
          4.84770000e+05,   1.55777000e+05],
       [  6.71000000e+02,   2.38300000e+03,   6.59000000e+03,
          1.12700000e+03,   6.71000000e+02],
       [  2.52000000e+02,   1.12800000e+03,   3.26600000e+03,
          5.25000000e+02,   4.18000000e+02],
       [ -1.25000000e+02,   1.49800000e+03,   5.74300000e+03,
          9.24000000e+02,   1.02700000e+03],
       [  4.41800000e+03,   1.32640000e+04,   3.60410000e+04,
          6.56700000e+03,   4.37000000e+03],
       [  6.85185000e+05,   1.6941000

In [126]:
SUM_Bid1_13

array([[  7.60000000e+01,   2.32000000e+02,   1.87280000e+04,
          7.57000000e+02,   9.20000000e+01],
       [  2.30000000e+01,   3.20000000e+01,   3.08100000e+03,
          2.78000000e+02,   7.40000000e+01],
       [ -1.01000000e+02,   8.00000000e+01,  -4.88000000e+03,
          5.03000000e+02,   5.80000000e+01],
       [  2.95000000e+02,   4.66000000e+02,   3.47580000e+04,
          1.04600000e+03,   2.26000000e+02],
       [  6.47080000e+04,   1.89330000e+05,   1.54237720e+07,
          4.81045000e+05,   6.03600000e+03],
       [  4.30000000e+01,   3.34000000e+02,   1.45650000e+04,
          5.73000000e+02,   1.90000000e+02],
       [  1.20000000e+01,   7.10000000e+01,   1.62400000e+03,
          4.11000000e+02,   1.60000000e+02],
       [ -6.60000000e+01,   1.80000000e+01,   1.00500000e+03,
          2.30000000e+01,   1.39000000e+02],
       [  1.85000000e+02,   9.02000000e+02,   1.90880000e+04,
          1.51900000e+03,   5.01000000e+02],
       [  5.53340000e+04,   2.4807100

In [127]:
SUM_Bid2_13

array([[  7.60000000e+01,   2.32000000e+02,   1.87280000e+04,
          7.57000000e+02,   9.20000000e+01],
       [  1.80000000e+01,   1.00000000e+01,   1.29200000e+03,
          2.39000000e+02,   6.40000000e+01],
       [ -1.03000000e+02,   8.40000000e+01,  -5.73800000e+03,
          6.97000000e+02,   1.26000000e+02],
       [  3.91000000e+02,   7.20000000e+02,   5.62790000e+04,
          1.89200000e+03,   3.33000000e+02],
       [  7.80070000e+04,   2.00195000e+05,   1.62795340e+07,
          5.48083000e+05,   1.88060000e+04],
       [  4.30000000e+01,   3.34000000e+02,   1.45650000e+04,
          5.73000000e+02,   1.90000000e+02],
       [  9.00000000e+00,   1.20000000e+01,   8.19000000e+02,
          3.57000000e+02,   1.03000000e+02],
       [ -6.90000000e+01,  -7.30000000e+01,   1.52000000e+03,
          3.18000000e+02,   2.90000000e+02],
       [  2.40000000e+02,   1.28300000e+03,   3.44280000e+04,
          2.23600000e+03,   7.00000000e+02],
       [  1.09391000e+05,   2.9052400

In [128]:
SUM_Bid3_13

array([[  7.60000000e+01,   2.32000000e+02,   1.87280000e+04,
          7.57000000e+02,   9.20000000e+01],
       [  8.00000000e+00,   1.00000000e+01,   5.21000000e+02,
          2.11000000e+02,   4.40000000e+01],
       [ -1.35000000e+02,   9.40000000e+01,  -6.04800000e+03,
          8.82000000e+02,   1.87000000e+02],
       [  5.07000000e+02,   9.52000000e+02,   7.51630000e+04,
          2.65700000e+03,   4.25000000e+02],
       [  1.05299000e+05,   2.00265000e+05,   1.66102150e+07,
          5.91859000e+05,   1.15110000e+04],
       [  4.30000000e+01,   3.34000000e+02,   1.45650000e+04,
          5.73000000e+02,   1.90000000e+02],
       [  2.00000000e+00,   1.00000000e+01,   5.26000000e+02,
          2.41000000e+02,   5.50000000e+01],
       [ -9.50000000e+01,  -6.90000000e+01,   1.83000000e+03,
          4.86000000e+02,   3.83000000e+02],
       [  3.11000000e+02,   1.62400000e+03,   4.91150000e+04,
          2.82700000e+03,   8.90000000e+02],
       [  1.58616000e+05,   2.9246800

In [129]:
SUM_Bid1_16

array([[  2.00000000e+00,   5.65000000e+02,   2.03760000e+04,
          2.05200000e+03,   2.39000000e+02],
       [  2.00000000e+00,   2.71000000e+02,   1.15910000e+04,
          1.00100000e+03,   7.10000000e+01],
       [  2.00000000e+00,  -1.14000000e+02,   5.61620000e+03,
         -7.20000000e+01,  -3.57000000e+02],
       [  4.00000000e+00,   5.59100000e+03,   3.48820000e+04,
          4.07900000e+03,   1.25000000e+03],
       [  1.52985000e+05,   2.86746000e+05,   1.12475190e+07,
          1.18168000e+06,   3.05760000e+04],
       [  7.00000000e+00,   1.11100000e+03,   1.78990000e+04,
          1.64300000e+03,   1.58000000e+02],
       [  7.00000000e+00,   6.37000000e+02,   9.39500000e+03,
          8.71000000e+02,   6.50000000e+01],
       [  7.00000000e+00,   2.77000000e+02,   1.30440000e+03,
          1.81000000e+02,  -7.50000000e+01],
       [  2.00000000e+01,   2.64300000e+03,   3.97450000e+04,
          3.70900000e+03,   6.48000000e+02],
       [  8.92500000e+04,   6.2967800

In [130]:
SUM_Bid2_16

array([[  2.00000000e+00,   5.65000000e+02,   2.03760000e+04,
          2.05200000e+03,   2.39000000e+02],
       [  0.00000000e+00,   2.06000000e+02,   5.60900000e+03,
          5.61000000e+02,   3.30000000e+01],
       [  4.00000000e+00,   7.00000000e+01,   7.14960000e+03,
          4.99000000e+02,  -3.32000000e+02],
       [  6.00000000e+00,   8.88100000e+03,   5.89080000e+04,
          6.26400000e+03,   1.48900000e+03],
       [  1.87200000e+05,   3.80714000e+05,   1.48509420e+07,
          1.52306700e+06,   2.29730000e+04],
       [  7.00000000e+00,   1.11100000e+03,   1.78990000e+04,
          1.64300000e+03,   1.58000000e+02],
       [  0.00000000e+00,   4.03000000e+02,   3.98500000e+03,
          3.89000000e+02,   4.70000000e+01],
       [  2.00000000e+00,   5.54000000e+02,  -2.76000000e+01,
          5.68000000e+02,  -4.60000000e+01],
       [  2.70000000e+01,   3.80900000e+03,   6.02580000e+04,
          5.46000000e+03,   8.06000000e+02],
       [  1.06200000e+05,   8.1837000

In [131]:
a = 0
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,307.000000,323.000000,3185.000000,296.000000,22.000000
1,0.693811,0.894737,0.922135,0.902027,1.000000
2,-0.140065,0.786378,0.982104,0.864865,1.000000
3,2.957655,1.721362,2.231711,1.591216,1.000000
4,354.084691,342.928793,406.663108,193.128378,98.818182


In [132]:
a = 5
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,395.000000,501.000000,977.000000,119.000000,4.0
1,0.878481,0.874251,0.925281,0.983193,1.0
2,0.572152,0.708583,0.963153,1.310924,1.0
3,2.283544,2.031936,2.327533,2.201681,2.0
4,263.217722,300.856287,310.683726,168.042017,64.0


In [133]:
a = 10
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,339.000000,488.000000,832.000000,116.000000,0.0
1,0.949853,0.924180,0.918269,1.000000,NaN
2,0.893805,0.842213,0.984375,1.431034,NaN
3,2.005900,1.942623,3.387019,2.353448,NaN
4,204.454277,285.819672,378.975962,205.853448,NaN


In [134]:
a = 15
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,187.000000,315.000000,549.000000,84.000000,0.0
1,0.925134,0.831746,0.903461,1.000000,NaN
2,0.877005,0.355556,0.808743,1.130952,NaN
3,2.550802,2.619048,2.795993,1.928571,NaN
4,266.139037,328.688889,331.805100,188.404762,NaN


In [135]:
a = 20
pd.DataFrame([SUM_Bid1_15[a],SUM_Bid1_15[a+1]/SUM_Bid1_15[a],SUM_Bid1_15[a+2]/SUM_Bid1_15[a],SUM_Bid1_15[a+3]/SUM_Bid1_15[a],SUM_Bid1_15[a+4]/SUM_Bid1_15[a]])

,0,1,2,3,4
0,361.000000,890.000000,1432.000000,278.000000,0.0
1,0.972299,0.849438,0.935754,0.982014,NaN
2,0.961219,0.515730,0.851955,1.237410,NaN
3,2.385042,3.588764,3.057961,2.654676,NaN
4,227.216066,420.226966,412.754190,287.435252,NaN


In [136]:
a = 0
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,307.000000,323.000000,3185.000000,296.000000,22.000000
1,0.693811,0.860681,0.889168,0.868243,1.000000
2,0.553746,1.551084,1.876295,1.729730,2.000000
3,3.990228,2.919505,3.258399,2.820946,2.000000
4,414.716612,417.244582,510.149451,299.263514,181.818182


In [137]:
a = 5
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,395.000000,501.000000,977.000000,119.000000,4.0
1,0.873418,0.818363,0.861822,0.957983,1.0
2,1.425316,1.433134,1.792221,2.294118,2.0
3,3.382278,3.337325,3.404299,3.310924,3.0
4,355.225316,400.506986,419.856704,248.176471,85.0


In [138]:
a = 10
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,339.000000,488.000000,832.000000,116.000000,0.0
1,0.946903,0.838115,0.750000,0.982759,NaN
2,1.828909,1.592213,1.770433,2.431034,NaN
3,3.147493,3.491803,4.497596,3.439655,NaN
4,332.407080,402.108607,482.161058,298.293103,NaN


In [139]:
a = 15
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,187.000000,315.000000,549.000000,84.000000,0.0
1,0.925134,0.755556,0.779599,1.000000,NaN
2,1.802139,0.952381,1.571949,2.130952,NaN
3,3.668449,4.447619,3.969035,3.107143,NaN
4,370.000000,447.609524,429.621129,361.666667,NaN


In [140]:
a = 20
pd.DataFrame([SUM_Bid2_15[a],SUM_Bid2_15[a+1]/SUM_Bid2_15[a],SUM_Bid2_15[a+2]/SUM_Bid2_15[a],SUM_Bid2_15[a+3]/SUM_Bid2_15[a],SUM_Bid2_15[a+4]/SUM_Bid2_15[a]])

,0,1,2,3,4
0,361.000000,890.000000,1432.000000,278.000000,0.0
1,0.972299,0.768539,0.807263,0.982014,NaN
2,1.933518,1.156180,1.580307,2.219424,NaN
3,3.465374,5.178652,4.216480,4.129496,NaN
4,325.626039,544.113483,528.382682,511.320144,NaN


In [141]:
a = 0
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,307.000000,323.000000,3185.000000,296.000000,22.000000
1,0.687296,0.476780,0.698587,0.868243,1.000000
2,1.224756,1.182663,2.298901,2.597973,3.000000
3,5.055375,5.071207,4.322763,3.837838,3.000000
4,478.934853,661.696594,615.751962,397.125000,181.818182


In [142]:
a = 5
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,395.000000,501.000000,977.000000,119.000000,4.0
1,0.863291,0.453094,0.755374,0.957983,1.0
2,2.248101,0.780439,2.507677,3.252101,3.0
3,4.443038,5.419162,4.486182,4.680672,4.0
4,405.108861,656.650699,519.662231,378.193277,107.0


In [143]:
a = 10
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,339.000000,488.000000,832.000000,116.000000,0.0
1,0.938053,0.461066,0.585337,0.982759,NaN
2,2.749263,0.926230,2.225962,3.413793,NaN
3,4.165192,5.678279,5.632212,4.887931,NaN
4,391.569322,690.176230,591.039663,477.922414,NaN


In [144]:
a = 15
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,187.000000,315.000000,549.000000,84.000000,0.0
1,0.925134,0.711111,0.646630,1.000000,NaN
2,2.727273,1.577778,2.078324,3.130952,NaN
3,4.673797,5.774603,5.120219,4.369048,NaN
4,435.385027,564.301587,566.213115,545.702381,NaN


In [145]:
a = 20
pd.DataFrame([SUM_Bid3_15[a],SUM_Bid3_15[a+1]/SUM_Bid3_15[a],SUM_Bid3_15[a+2]/SUM_Bid3_15[a],SUM_Bid3_15[a+3]/SUM_Bid3_15[a],SUM_Bid3_15[a+4]/SUM_Bid3_15[a]])

,0,1,2,3,4
0,361.000000,890.000000,1432.000000,278.000000,0.0
1,0.950139,0.680899,0.729749,0.982014,NaN
2,2.795014,1.774157,2.262570,3.201439,NaN
3,4.565097,6.259551,5.307961,5.230216,NaN
4,400.470914,609.579775,612.990223,615.352518,NaN


In [146]:
a = 0
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,681.000000,2272.000000,5032.000000,814.000000,597.000000
1,0.897210,0.878961,0.853537,0.845209,0.812395
2,0.643172,0.638204,0.466614,0.552826,0.467337
3,2.008811,2.156250,2.588037,3.287469,3.716918
4,450.215859,251.857835,294.426868,350.970516,132.520938


In [147]:
a = 5
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,671.000000,2383.000000,6590.000000,1127.000000,671.000000
1,0.789866,0.812841,0.795448,0.741792,0.777943
2,-0.216095,0.295846,0.254476,0.110027,0.424739
3,3.339791,2.799832,2.859636,3.443656,4.159463
4,611.926975,374.715485,361.462671,383.092280,159.763040


In [148]:
a = 10
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,658.000000,2746.000000,7926.000000,1305.000000,673.000000
1,0.688450,0.764385,0.754353,0.704981,0.827637
2,-0.709726,-0.032775,0.178526,-0.055172,0.546805
3,4.077508,3.264385,3.122761,3.855172,3.998514
4,663.933131,435.257101,411.616326,429.982375,172.019316


In [149]:
a = 15
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,532.000000,2028.000000,6173.000000,1019.000000,333.000000
1,0.569549,0.709566,0.684918,0.687929,0.717718
2,-1.701128,-0.118343,-0.177709,-0.198234,0.240240
3,5.469925,3.441815,3.489227,4.112856,4.795796
4,736.625940,491.636095,466.537826,477.061825,171.651652


In [150]:
a = 20
pd.DataFrame([SUM_Bid1_11[a],SUM_Bid1_11[a+1]/SUM_Bid1_11[a],SUM_Bid1_11[a+2]/SUM_Bid1_11[a],SUM_Bid1_11[a+3]/SUM_Bid1_11[a],SUM_Bid1_11[a+4]/SUM_Bid1_11[a]])

,0,1,2,3,4
0,1053.000000,3431.000000,9100.000000,1812.000000,373.000000
1,0.448243,0.637715,0.600549,0.669426,0.734584
2,-2.925926,-0.528709,-0.590769,-0.530905,0.018767
3,6.446344,3.831245,3.998132,3.969647,4.495979
4,723.368471,512.988342,529.479231,487.352649,138.825737


In [151]:
a = 0
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,681.000000,2272.000000,5032.000000,814.000000,597.000000
1,0.681351,0.789173,0.756359,0.748157,0.775544
2,1.042584,1.137324,1.065580,1.055283,1.167504
3,3.665198,3.519366,3.932432,4.660934,5.087102
4,671.098385,424.057658,435.360692,475.216216,234.668342


In [152]:
a = 5
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,671.000000,2383.000000,6590.000000,1127.000000,671.000000
1,0.576751,0.664289,0.657815,0.564330,0.719821
2,-0.514158,0.484264,0.658270,0.468500,1.061103
3,5.472429,4.337809,4.222914,4.749778,5.429210
4,877.450075,569.543433,517.181032,561.022183,205.684054


In [153]:
a = 10
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,658.000000,2746.000000,7926.000000,1305.000000,673.000000
1,0.519757,0.609978,0.578224,0.501916,0.774146
2,-0.866261,0.191916,0.424552,0.214559,1.267459
3,5.863222,4.707210,4.444991,5.156322,5.234770
4,848.943769,610.981063,567.324249,614.866667,222.268945


In [154]:
a = 15
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,532.000000,2028.000000,6173.000000,1019.000000,333.000000
1,0.387218,0.496548,0.499109,0.451423,0.642643
2,-2.287594,-0.177022,0.009882,0.040236,0.849850
3,7.411654,5.025148,4.787462,5.289500,5.939940
4,932.073308,654.364892,622.114855,658.031403,191.702703


In [155]:
a = 20
pd.DataFrame([SUM_Bid2_11[a],SUM_Bid2_11[a+1]/SUM_Bid2_11[a],SUM_Bid2_11[a+2]/SUM_Bid2_11[a],SUM_Bid2_11[a+3]/SUM_Bid2_11[a],SUM_Bid2_11[a+4]/SUM_Bid2_11[a]])

,0,1,2,3,4
0,1053.000000,3431.000000,9100.000000,1812.000000,373.000000
1,0.303894,0.450015,0.428022,0.513797,0.683646
2,-3.203229,-0.608569,-0.457033,-0.193157,0.640751
3,8.144349,5.379773,5.345604,5.135210,5.766756
4,841.364672,669.260565,682.354615,614.055740,175.766756


In [156]:
a = 0
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,681.000000,2272.000000,5032.000000,814.000000,597.000000
1,0.502203,0.624560,0.577901,0.654791,0.713568
2,1.547724,1.415933,1.319555,1.613022,1.845896
3,4.716593,4.801937,5.166335,5.730958,6.107203
4,801.387665,581.121039,588.632353,595.540541,260.932998


In [157]:
a = 5
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,671.000000,2383.000000,6590.000000,1127.000000,671.000000
1,0.375559,0.473353,0.495599,0.465839,0.622951
2,-0.186289,0.628619,0.871472,0.819876,1.530551
3,6.584203,5.566093,5.469044,5.826974,6.512668
4,1021.140089,710.910617,657.741123,653.744454,222.675112


In [158]:
a = 10
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,658.000000,2746.000000,7926.000000,1305.000000,673.000000
1,0.360182,0.442098,0.458617,0.351724,0.671620
2,-0.571429,0.234523,0.727101,0.358621,1.751857
3,6.968085,6.025856,5.554252,6.228352,6.334324
4,996.006079,744.319738,673.741862,716.855939,232.203566


In [159]:
a = 15
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,532.000000,2028.000000,6173.000000,1019.000000,333.000000
1,0.274436,0.354043,0.396242,0.319921,0.465465
2,-2.167293,-0.049803,0.298882,0.187439,0.942943
3,8.592105,6.207101,5.869755,6.355250,7.090090
4,1067.043233,761.539448,711.534748,749.997056,136.702703


In [160]:
a = 20
pd.DataFrame([SUM_Bid3_11[a],SUM_Bid3_11[a+1]/SUM_Bid3_11[a],SUM_Bid3_11[a+2]/SUM_Bid3_11[a],SUM_Bid3_11[a+3]/SUM_Bid3_11[a],SUM_Bid3_11[a+4]/SUM_Bid3_11[a]])

,0,1,2,3,4
0,1053.000000,3431.000000,9100.000000,1812.000000,373.000000
1,0.249763,0.292918,0.336703,0.387969,0.597855
2,-2.966762,-0.586418,-0.203736,0.057395,0.989276
3,9.170940,6.559604,6.429011,6.198124,6.903485
4,894.863248,780.984553,750.077473,700.118653,194.367292


In [161]:
a = 0
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,76.000000,232.000000,18728.000000,757.000000,92.000000
1,0.302632,0.137931,0.164513,0.367239,0.804348
2,-1.328947,0.344828,-0.260572,0.664465,0.630435
3,3.881579,2.008621,1.855938,1.381770,2.456522
4,851.421053,816.077586,823.567493,635.462351,65.608696


In [162]:
a = 5
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,43.000000,334.000000,14565.000000,573.000000,190.000000
1,0.279070,0.212575,0.111500,0.717277,0.842105
2,-1.534884,0.053892,0.069001,0.040140,0.731579
3,4.302326,2.700599,1.310539,2.650960,2.636842
4,1286.837209,742.727545,839.450875,410.188482,221.463158


In [163]:
a = 10
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,89.000000,560.000000,6979.000000,784.000000,147.000000
1,0.269663,0.178571,0.286717,0.744898,0.571429
2,-0.910112,0.366071,0.044419,0.082908,0.612245
3,4.235955,2.730357,2.149878,3.173469,3.646259
4,1302.404494,791.312500,727.229546,496.503827,101.394558


In [164]:
a = 15
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,79.000000,459.000000,5920.000000,476.000000,106.000000
1,0.379747,0.126362,0.301351,0.766807,0.726415
2,-1.215190,0.122004,-0.098986,0.443277,0.688679
3,4.253165,3.041394,2.440203,2.894958,3.386792
4,909.392405,817.095861,735.653716,455.544118,214.320755


In [165]:
a = 20
pd.DataFrame([SUM_Bid1_13[a],SUM_Bid1_13[a+1]/SUM_Bid1_13[a],SUM_Bid1_13[a+2]/SUM_Bid1_13[a],SUM_Bid1_13[a+3]/SUM_Bid1_13[a],SUM_Bid1_13[a+4]/SUM_Bid1_13[a]])

,0,1,2,3,4
0,130.000000,1048.000000,18939.000000,548.000000,146.000000
1,0.153846,0.166985,0.200961,0.531022,0.746575
2,-2.361538,-0.687023,-0.053118,-2.093066,0.712329
3,5.353846,3.208015,2.065949,5.572993,3.061644
4,1025.661538,794.950382,808.758593,609.191606,309.438356


In [166]:
a = 0
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,76.000000,232.000000,18728.000000,757.000000,92.000000
1,0.236842,0.043103,0.068988,0.315720,0.695652
2,-1.355263,0.362069,-0.306386,0.920740,1.369565
3,5.144737,3.103448,3.005073,2.499339,3.619565
4,1026.407895,862.909483,869.261747,724.019815,204.413043


In [167]:
a = 5
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,43.000000,334.000000,14565.000000,573.000000,190.000000
1,0.209302,0.035928,0.056231,0.623037,0.542105
2,-1.604651,-0.218563,0.104360,0.554974,1.526316
3,5.581395,3.841317,2.363749,3.902269,3.684211
4,2543.976744,869.832335,870.374597,599.040140,187.526316


In [168]:
a = 10
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,89.000000,560.000000,6979.000000,784.000000,147.000000
1,0.179775,0.067857,0.199169,0.556122,0.462585
2,-1.089888,0.398214,0.155753,0.307398,1.149660
3,5.606742,3.792857,3.261499,4.651786,4.653061
4,1547.146067,865.251786,807.494483,709.463010,149.659864


In [169]:
a = 15
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,79.000000,459.000000,5920.000000,476.000000,106.000000
1,0.037975,0.069717,0.184122,0.560924,0.424528
2,-2.544304,0.111111,0.009291,0.638655,1.292453
3,6.645570,4.104575,3.562500,4.392857,4.443396
4,1564.911392,858.172113,825.241385,674.563025,193.113208


In [170]:
a = 20
pd.DataFrame([SUM_Bid2_13[a],SUM_Bid2_13[a+1]/SUM_Bid2_13[a],SUM_Bid2_13[a+2]/SUM_Bid2_13[a],SUM_Bid2_13[a+3]/SUM_Bid2_13[a],SUM_Bid2_13[a+4]/SUM_Bid2_13[a]])

,0,1,2,3,4
0,130.000000,1048.000000,18939.000000,548.000000,146.000000
1,0.023077,0.129771,0.097629,0.383212,0.164384
2,-2.861538,-0.640267,0.016949,-1.895985,1.157534
3,6.892308,4.309160,3.095887,6.872263,4.061644
4,1726.607692,831.361641,860.858599,782.744526,44.643836


In [171]:
a = 0
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,76.000000,232.000000,18728.000000,757.000000,92.000000
1,0.105263,0.043103,0.027819,0.278732,0.478261
2,-1.776316,0.405172,-0.322939,1.165125,2.032609
3,6.671053,4.103448,4.013402,3.509908,4.619565
4,1385.513158,863.211207,886.918785,781.848085,125.119565


In [172]:
a = 5
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,43.000000,334.000000,14565.000000,573.000000,190.000000
1,0.046512,0.029940,0.036114,0.420593,0.289474
2,-2.209302,-0.206587,0.125644,0.848168,2.015789
3,7.232558,4.862275,3.372125,4.933682,4.684211
4,3688.744186,875.652695,884.053278,732.888307,103.989474


In [173]:
a = 10
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,89.000000,560.000000,6979.000000,784.000000,147.000000
1,0.011236,0.023214,0.129388,0.270408,0.299320
2,-1.730337,0.401786,0.257917,0.415816,1.537415
3,7.258427,4.805357,4.281272,5.727041,5.653061
4,1932.707865,880.891071,855.047571,814.390306,71.904762


In [174]:
a = 15
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,79.000000,459.000000,5920.000000,476.000000,106.000000
1,0.000000,0.013072,0.092230,0.327731,0.113208
2,-2.696203,0.106754,0.111149,0.869748,1.575472
3,7.784810,5.104575,4.579730,5.418067,5.443396
4,1970.886076,889.074074,860.951520,769.514706,34.990566


In [175]:
a = 20
pd.DataFrame([SUM_Bid3_13[a],SUM_Bid3_13[a+1]/SUM_Bid3_13[a],SUM_Bid3_13[a+2]/SUM_Bid3_13[a],SUM_Bid3_13[a+3]/SUM_Bid3_13[a],SUM_Bid3_13[a+4]/SUM_Bid3_13[a]])

,0,1,2,3,4
0,130.000000,1048.000000,18939.000000,548.000000,146.000000
1,0.000000,0.001908,0.046676,0.200730,0.068493
2,-2.938462,-0.745229,0.044934,-1.713504,1.239726
3,7.961538,5.312977,4.099372,7.905109,5.061644
4,1841.538462,898.415076,886.174772,842.583942,20.952055


In [176]:
a = 0
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,2.0,565.000000,20376.000000,2052.000000,239.000000
1,1.0,0.479646,0.568856,0.487817,0.297071
2,1.0,-0.201770,0.275628,-0.035088,-1.493724
3,2.0,9.895575,1.711916,1.987817,5.230126
4,76492.5,507.515044,551.998380,575.867446,127.933054


In [177]:
a = 5
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,7.000000,1111.000000,17899.000000,1643.000000,158.000000
1,1.000000,0.573357,0.524890,0.530128,0.411392
2,1.000000,0.249325,0.072876,0.110164,-0.474684
3,2.857143,2.378938,2.220515,2.257456,4.101266
4,12750.000000,566.766877,609.057210,627.395009,178.297468


In [178]:
a = 10
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,6.000000,1628.000000,18016.000000,1319.000000,127.000000
1,1.000000,0.550983,0.446270,0.575436,0.629921
2,1.000000,-0.151720,-0.091519,0.277483,0.015748
3,2.166667,2.674447,2.498057,2.257771,3.858268
4,7912.000000,588.132064,664.859514,549.219864,184.692913


In [179]:
a = 15
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,1.0,1156.000000,12321.000000,1145.000000,103.000000
1,1.0,0.438581,0.431702,0.451528,0.640777
2,1.0,-0.208478,-0.170522,0.044541,0.087379
3,2.0,2.955017,2.688093,2.519651,3.932039
4,34178.0,643.929931,683.798718,601.392140,207.300971


In [180]:
a = 20
pd.DataFrame([SUM_Bid1_16[a],SUM_Bid1_16[a+1]/SUM_Bid1_16[a],SUM_Bid1_16[a+2]/SUM_Bid1_16[a],SUM_Bid1_16[a+3]/SUM_Bid1_16[a],SUM_Bid1_16[a+4]/SUM_Bid1_16[a]])

,0,1,2,3,4
0,53.000000,3170.000000,23496.000000,1191.000000,216.000000
1,1.000000,0.292744,0.361296,0.314022,0.916667
2,1.000000,-0.796215,-0.633129,-0.104114,0.824074
3,2.792453,4.207571,2.794646,2.736356,2.726852
4,1130.132075,675.446057,699.047668,702.225861,303.893519


In [181]:
a = 0
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,2.0,565.000000,20376.000000,2052.000000,239.000000
1,0.0,0.364602,0.275275,0.273392,0.138075
2,2.0,0.123894,0.350883,0.243177,-1.389121
3,3.0,15.718584,2.891048,3.052632,6.230126
4,93600.0,673.830088,728.844817,742.235380,96.121339


In [182]:
a = 5
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,7.000000,1111.000000,17899.000000,1643.000000,158.000000
1,0.000000,0.362736,0.222638,0.236762,0.297468
2,0.285714,0.498650,-0.001542,0.345709,-0.291139
3,3.857143,3.428443,3.366557,3.323189,5.101266
4,15171.428571,736.606661,777.050003,787.720633,180.987342


In [183]:
a = 10
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,6.000000,1628.000000,18016.000000,1319.000000,127.000000
1,0.000000,0.326167,0.194549,0.298711,0.606299
2,1.666667,0.080467,-0.101743,0.502654,0.622047
3,3.166667,3.703317,3.587811,3.329037,4.858268
4,8400.000000,744.773342,798.924512,733.037149,242.110236


In [184]:
a = 15
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,1.0,1156.000000,12321.000000,1145.000000,103.000000
1,0.0,0.250000,0.154289,0.265502,0.640777
2,2.0,-0.113322,-0.202873,0.197380,0.728155
3,3.0,3.975779,3.761221,3.620961,4.932039
4,34200.0,763.416090,814.298109,737.790393,293.300971


In [185]:
a = 20
pd.DataFrame([SUM_Bid2_16[a],SUM_Bid2_16[a+1]/SUM_Bid2_16[a],SUM_Bid2_16[a+2]/SUM_Bid2_16[a],SUM_Bid2_16[a+3]/SUM_Bid2_16[a],SUM_Bid2_16[a+4]/SUM_Bid2_16[a]])

,0,1,2,3,4
0,53.000000,3170.000000,23496.000000,1191.000000,216.000000
1,0.000000,0.116404,0.111764,0.214945,0.879630
2,0.716981,-0.821136,-0.767824,0.064652,1.703704
3,3.792453,5.688013,3.846101,3.748111,3.726852
4,1341.509434,787.931230,806.263321,786.382872,398.439815


In [186]:
a = 0
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,2.0,565.000000,20376.000000,2052.000000,239.000000
1,0.0,0.182301,0.106989,0.095517,0.133891
2,2.0,0.208850,0.405291,0.318226,-1.255230
3,4.0,20.502655,3.909109,4.207602,7.230126
4,94050.0,744.964602,796.932862,805.858187,98.757322


In [187]:
a = 5
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,7.000000,1111.000000,17899.000000,1643.000000,158.000000
1,0.000000,0.096310,0.096374,0.086427,0.284810
2,0.285714,0.456346,0.033924,0.449178,-0.006329
3,4.857143,4.429343,4.387508,4.357273,6.101266
4,15428.571429,865.545455,836.548746,847.940353,197.170886


In [188]:
a = 10
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,6.000000,1628.000000,18016.000000,1319.000000,127.000000
1,0.000000,0.057125,0.073990,0.144049,0.598425
2,1.666667,-0.074324,-0.134825,0.648218,1.220472
3,4.166667,4.703317,4.618450,4.404852,5.858268
4,8550.000000,875.785012,859.226188,805.161486,330.826772


In [189]:
a = 15
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,1.0,1156.000000,12321.000000,1145.000000,103.000000
1,0.0,0.088235,0.065741,0.160699,0.601942
2,2.0,-0.064879,-0.233552,0.440175,1.330097
3,4.0,4.975779,4.791170,4.661135,5.932039
4,37800.0,859.988754,862.550605,800.602620,355.844660


In [190]:
a = 20
pd.DataFrame([SUM_Bid3_16[a],SUM_Bid3_16[a+1]/SUM_Bid3_16[a],SUM_Bid3_16[a+2]/SUM_Bid3_16[a],SUM_Bid3_16[a+3]/SUM_Bid3_16[a],SUM_Bid3_16[a+4]/SUM_Bid3_16[a]])

,0,1,2,3,4
0,53.000000,3170.000000,23496.000000,1191.000000,216.000000
1,0.000000,0.024921,0.055158,0.160369,0.800926
2,0.716981,-0.843849,-0.796595,0.268682,2.504630
3,4.792453,7.140379,4.887215,4.754828,4.726852
4,1630.188679,833.238170,840.080482,835.130982,477.768519
